In [437]:
import pandas as pd
import numpy as np
import pyreadstat
data = pd.read_table('PartD_Prescriber_PUF_NPI_17.txt', header=None)
data.columns=data.loc[0]
data=data.drop(0,axis=0)
data.head()


C:\Users\alici\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,10,11,17,18,19,20,21,23,24,25,26,28,30,31,33,34,36,37,39,40,42,43,45,46,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,64,65,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,beneficiary_male_count,beneficiary_race_white_count,beneficiary_race_black_count,beneficiary_race_asian_pi_count,beneficiary_race_hispanic_count,beneficiary_race_nat_ind_count,beneficiary_race_other_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score
1,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,...,92,220,14,0,0,0,0,143,91,2.1685
2,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,...,92,195,58,NaN,NaN,0,NaN,143,133,1.8029
3,1003000167,ESCOBAR,JULIO,E,DDS,M,I,5 PINE CONE RD,NaN,DAYTON,...,17,NaN,0,0,NaN,0,0,NaN,NaN,1.0598
4,1003000175,REYES-VASQUEZ,BELINDA,NaN,D.D.S.,F,I,322 N AZUSA AVE STE 202,NaN,LA PUENTE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1003000282,BLAKEMORE,ROSIE,K,FNP,F,I,TENNESSEE PRISON FOR WOMEN,3881 STEWARTS LANE,NASHVILLE,...,NaN,NaN,NaN,0,NaN,0,0,NaN,NaN,4.5148


In 2017, what was the average number of beneficiaries per provider? Due to the suppression of data for those with few beneficiaries, we can only include those with more than 10 beneficiaries.

In [440]:
print(data.bene_count.astype(float).mean())

158.3494585173676


For each provider, estimate the length of the average prescription from the total_day_supply and total_claim_count. What is the median, in days, of the distribution of this value across all providers?

In [441]:
p_length=data.total_day_supply.astype(float)/data.total_claim_count.astype(float)
print('median prescription length : ', p_length.median())

median prescription length :  29.7125748502994


Work out for each Specialty the fraction of drug claims that are for brand-name drugs. Include only providers for whom the relevant information has not been suppressed, and consider only specialties with at least 1000 total claims. What is the standard deviation of these fractions?

In [442]:
#convert columns with counts to numerical values from strings
data.brand_claim_count=data.brand_claim_count.astype(float)
data.generic_claim_count=data.generic_claim_count.astype(float)

#Filter by specialties with at least 1000 total claims
#specialty=data[(data.brand_claim_count + data.generic_claim_count) >=1000]

#group specialty, brand claims and generic claims by specialty
specialty=specialty.loc[:,['specialty_description','brand_claim_count','generic_claim_count']]
grouped=pd.DataFrame(specialty.groupby('specialty_description'))
specialty_names=grouped[0]

#For each specialty, calculate the fraction of drug claims that are for brand-name
specialtyratios=np.zeros(len(specialty_names))
tot_claims=np.zeros(len(specialty_names))
for i in range(0,len(specialty_names)):
    
    #sum the generic, brand name and total claims
    tot_brand=grouped[1][i].brand_claim_count.sum()
    tot_generic=grouped[1][i].generic_claim_count.sum()
    tot_claims[i]=tot_brand+tot_generic
    
    #fraction of drug claims that are for brand-name
    specialtyratios[i]=tot_brand/(tot_brand+tot_generic)

#filter by total claims >= 1000
specialtyratios100=specialtyratios[tot_claims>=1000]
print('std of brand-name ratios: ', np.std(specialtyratios100))


std of brand-name ratios:  0.09790532162619664


Find the ratio of beneficiaries with opioid prescriptions to beneficiaries with antibiotics prescriptions in each state. Assume that each beneficiary attends only a single provider. What is the difference between the largest and smallest ratios?

In [443]:
#convert columns with counts to numerical values from strings
data.opioid_claim_count=data.opioid_claim_count.astype(float)
data.antibiotic_claim_count=data.antibiotic_claim_count.astype(float)

#Group by state
state=data.loc[:,['nppes_provider_state','opioid_claim_count','antibiotic_claim_count']]
grouped=pd.DataFrame(state.groupby('nppes_provider_state'))
state_names=grouped[0]

#For each State find the ratio of beneficiaries with opioid prescriptions to beneficiaries with antibiotics prescriptions
num_states=len(state_names)
state_ratios=np.zeros(num_states)
for i in range(0,num_states):
    #sum the num of beneficiaries for opiod and antibiotic claims
    tot_opioid=grouped[1][i].opioid_claim_count.sum()
    tot_antibiotic=grouped[1][i].antibiotic_claim_count.sum()
    state_ratios[i]=tot_opioid/(tot_antibiotic)
print('Difference between max and min opiod-antibiotic ratios: ', np.max(state_ratios)-np.min(state_ratios))


Difference between max and min opiod-antibiotic ratios:  1.6361389612848332


For each provider where the relevant columns are not suppressed, work out the fraction of claims for beneficiaries age 65 and older, as well as the fraction of claims for beneficiaries with a low-income subsidy. What is the Pearson correlation coefficient between these values?

In [444]:
#convert columns with counts to numerical values from strings
data.total_claim_count_ge65=data.total_claim_count_ge65.astype(float)
data.total_claim_count=data.total_claim_count.astype(float)
data.lis_claim_count=data.lis_claim_count.astype(float)

# fraction of claims for beneficiaries age 65 and older
older65_ratio=data.total_claim_count_ge65/data.total_claim_count
lis_ratio=data.lis_claim_count/data.total_claim_count

#Calculate Pearson Correlation Coefficient
pc=np.corrcoef(pd.notnull(data.total_claim_count_ge65),pd.notnull(data.lis_claim_count))
print('Pearson Correlation Coefficient between 65+ claims and lis claims: ', pc[0,1])


Pearson Correlation Coefficient between 65+ claims and lis claims:  0.5807059616642903


Let's find which states have surprisingly high supply of opioids, conditioned on specialty. Work out the average length of an opioid prescription for each provider. For each (state, specialty) pair with at least 100 providers, calculate the average of this value across all providers. Then find the ratio of this value to an equivalent quantity calculated from providers in each specialty across all states. What is the largest such ratio?

In [445]:
#convert columns with counts to numerical values from strings
data.opioid_day_supply=data.opioid_day_supply.astype(float)

#Average opioid prescription length for each provider
opdata=data
opdata['opioid_presc_len']=opdata.opioid_day_supply.astype(float)/opdata.opioid_claim_count.astype(float)

#Filter data by (state, specialty) pairs with at least 100 providers
sspaired=pd.DataFrame({'numproviders': data.groupby(['nppes_provider_state', 'specialty_description']).size()})

In [447]:
gt100=sspaired[sspaired.numproviders[:]>100]
opdata=opdata.set_index(['nppes_provider_state','specialty_description'])
opdata=opdata.loc[gt100.index]
#opdata=opdata.reset_index()


In [448]:
grouped=opdata.groupby(['nppes_provider_state', 'specialty_description'])
groupedmean=grouped.mean()
groupedmean.opioid_presc_len

groupedspec=opdata.groupby(['specialty_description'])
groupedspec=groupedspec.mean()
groupedspec.opioid_presc_len

print(groupedmean.opioid_presc_len/groupedspec.opioid_presc_len)
print('The max ratio of avg opioid presc length per state and specialty to the avg precription length for each specialty is:',np.max(groupedmean.opioid_presc_len/groupedspec.opioid_presc_len))

nppes_provider_state  specialty_description
AK                    Dentist                  0.914285
                      Emergency Medicine       0.793522
                      Family Practice          1.035119
                      Internal Medicine        1.120950
                      Nurse Practitioner       1.228716
                                                 ...   
WY                    Dentist                  0.894691
                      Family Practice          0.954140
                      Internal Medicine        1.066584
                      Nurse Practitioner       1.060907
                      Physician Assistant      1.349154
Name: opioid_presc_len, Length: 1312, dtype: float64
The max ratio of avg opioid presc length per state and specialty to the avg precription length for each specialty is: 2.126102047010669


For each provider for whom the information is not suppressed, figure out the average cost per day of prescriptions in both 2016 and 2017. Use this to estimate the inflation rate for daily prescription costs per provider. What is the average inflation rate across all providers?

In [302]:
#Import 2016 data
data16 = pd.read_table('PartD_Prescriber_PUF_NPI_16.txt', header=None)
data16.columns=data16.loc[0]
data16=data16.drop(0,axis=0)

C:\Users\alici\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,10,11,17,18,19,20,21,23,24,25,26,28,30,31,33,34,36,37,39,40,42,43,45,46,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,64,65,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [553]:
#convert columns with counts to numerical values from strings
data.total_drug_cost=data.total_drug_cost.astype(float)
data.total_day_supply=data.total_day_supply.astype(float)
data16.total_drug_cost=data16.total_drug_cost.astype(float)
data16.total_day_supply=data16.total_day_supply.astype(float)

#average cost per day of prescriptions in both 2016 and 2017
data16['cost_per_day']=data16.total_drug_cost/data16.total_day_supply
data['cost_per_day']=data.total_drug_cost/data.total_day_supply

#Create combined data frame for 2016 and 2017 data for each provider, cost per day and day supply
cost=data.loc[:,['npi','cost_per_day']]
cost2=data16.loc[:,['npi','cost_per_day']]
cost=cost.set_index('npi').join(cost2.set_index('npi'),on='npi',rsuffix='16')
cost=cost.dropna()

#average inflation rate across all providers
infl_rate=(cost.cost_per_day-cost.cost_per_day16)/cost.cost_per_day16
print(infl_rate)

avg_infl_rate=infl_rate.mean()
print('average inflation rate across all providers:', avg_infl_rate)

npi
1003000126   -0.204274
1003000142   -0.121318
1003000167    0.334885
1003000282    2.651478
1003000407   -0.127496
                ...   
1992999825   -0.231220
1992999833    0.481279
1992999866    0.320788
1992999874   -0.102070
1992999882   -0.184960
Length: 1054153, dtype: float64
average inflation rate across all providers: 0.22753794094705018


Consider all providers with a defined specialty in both years. Find the fraction of providers who left each specialty between 2016 and 2017. What is the largest such fraction, when considering specialties with at least 1000 proviers in 2016? Note that some specialties have a fraction of 1 due to specialty name changes between 2016 and 2017; disregard these specialties in calculating your answer.

In [582]:
#add 2016 and 2017 data for specialty, removing rows with providers missing a defined specialty in both years

s=data.loc[:,['npi','specialty_description']]
s2=data16.loc[:,['npi','specialty_description']]
s=s2.set_index('npi').join(s.set_index('npi'),rsuffix='17')
s=s.dropna()

sleft=s[s.specialty_description!=s.specialty_description17]

provsleft=pd.DataFrame({'numproviders': sleft.groupby(['specialty_description']).size()})

provs16=pd.DataFrame({'numproviders': s2.groupby(['specialty_description']).size()})
provs16=provs16[provs16.numproviders>1000]

fracleft=provsleft/provs16
fracleft=fracleft.dropna()
fracleft.sort_values(by='numproviders')

,numproviders
specialty_description,
Podiatry,0.000196
Optometry,0.000350
Pharmacist,0.000489
Physician Assistant,0.001349
Dentist,0.001897
Nurse Practitioner,0.002540
Ophthalmology,0.002834
Urology,0.003922
Dermatology,0.004067
